## Knapsack Problem

### Problem Statement

We have a set of cars of different sizes and values and a parking lot at the dealership with a limited car capacity. How do we choose which cars to pack into the parking lot to maximize the total sale value?

### Define Variables and Parameters

* $N$ is the number of cars
* $s_i$ is the size of car $i$
* $v_i$ is the value of car $i$
* $x_i$ is the decision variables: $x_i = 1$ if car $i$ is chosen and $x_i = 0$ if not
* $c_i$ is the carbon tax of the car $i$
* $C$ is the total carbon tax amount the dealership paid. 
* $S$ is the capacity of the parking lot

### Identify Constraints and Make Assumptions

* Total area of the parked items is less than the parking lot capacity

In [1]:
# Import Packages
import numpy as np
import cvxpy as cp
import pandas as pd

In [16]:
# Data Loading
df = pd.read_csv('data/used_cars.csv')
print(list(df.columns))

['brand', 'model', 'model_year', 'milage', 'fuel_type', 'engine', 'transmission', 'ext_col', 'int_col', 'accident', 'clean_title', 'price']


In [17]:
# data cleaning
df = df.dropna()
df_new = df[df['fuel_type'] != "not supported"]
print(df_new['fuel_type'].unique())
df_new = df_new[df_new['fuel_type'] != '–']

df_new.drop(columns=["model_year", "milage","engine","transmission","ext_col","int_col","accident","clean_title"], inplace=True)

# Remove the dollar sign and convert to numeric
df_new['price'] = df_new['price'].str.replace(r'[$,]', '', regex=True).astype(float)
print(df_new.head())

['E85 Flex Fuel' 'Gasoline' 'Hybrid' 'Diesel' 'Plug-In Hybrid' '–']
      brand                            model      fuel_type    price
0      Ford  Utility Police Interceptor Base  E85 Flex Fuel  10300.0
1   Hyundai                     Palisade SEL       Gasoline  38005.0
3  INFINITI                 Q50 Hybrid Sport         Hybrid  15500.0
6      Audi             S3 2.0T Premium Plus       Gasoline  31000.0
7       BMW                           740 iL       Gasoline   7300.0


In [18]:
df_new["Depreciation Rate"] = [
    0.5 if type == "Diesel" else
    0.65 if type == "Gasoline" else
    0.7 if type == 'E85 Flex Fuel' else
    0.80 if type == "Hybrid" else
    0.95
    for type in df_new["fuel_type"]
]
df_new["Carbon Tax"] = [
    500 if type == "Diesel" else
    200 if type == "Gasoline" else
    150 if type == 'E85 Flex Fuel' else
    120 if type == "Hybrid" else
    50
    for type in df_new["fuel_type"]
]
df_new["Buying Price"] = df_new["price"]*df_new["Depreciation Rate"] + df_new["Carbon Tax"]
df_new["Profit"] = df_new["price"]-df_new["Buying Price"]
# Create the "imported" column, default hybrid and plug-in hybrid rows to 0
df_new['imported'] = np.where(df_new['fuel_type'].isin(['Plug-In Hybrid', 'Hybrid']), 0, np.nan)
# Randomly assign 0 or 1 to the rest of the rows
df_new['imported'] = df_new['imported'].apply(lambda x: np.random.choice([0, 1]) if np.isnan(x) else x)
print(df_new.head())

      brand                            model      fuel_type    price  \
0      Ford  Utility Police Interceptor Base  E85 Flex Fuel  10300.0   
1   Hyundai                     Palisade SEL       Gasoline  38005.0   
3  INFINITI                 Q50 Hybrid Sport         Hybrid  15500.0   
6      Audi             S3 2.0T Premium Plus       Gasoline  31000.0   
7       BMW                           740 iL       Gasoline   7300.0   

   Depreciation Rate  Carbon Tax  Buying Price    Profit  imported  
0               0.70         150       7360.00   2940.00       1.0  
1               0.65         200      24903.25  13101.75       1.0  
3               0.80         120      12520.00   2980.00       0.0  
6               0.65         200      20350.00  10650.00       0.0  
7               0.65         200       4945.00   2355.00       1.0  


In [30]:
# total number of cars 
N = len(df_new)
# value of cars varies
profit = df_new['Profit']
# size of the cars varies
s = np.random.randint(8,10,N)
# total size of all cars
size = np.sum(s)
# parking lot capacity
ParkCap = np.random.randint(size//2,2*size//3)

# Buying price for each car
b = df_new["Buying Price"]
# total amount of buying cost
buyCost = np.sum(b)
# total budget the dealership has
Budget = np.random.randint(buyCost//5,buyCost//4)

In [85]:
# approximation with greedy algorithm
fuel = df_new["fuel_type"].values
imported = df_new["imported"].values
profit = df_new['Profit'].values
budget = df_new['Buying Price'].values

global value_per_size 
value_per_size = profit/s

should_break=False

items = []
ev_record = []
import_record = []
global counter_import
counter_import = 0
global counter_EV
counter_EV = 0

EV_fee = 300
license_fee = 1000

In [80]:
# functions for the greedy algorithm:

# check the current profit for car i:
# takes in the index for car i, check if it is EV or Imported, and deduct profit accordingly.
def check_price(i):
    fuel_type = fuel[i]
    status = imported[i]
    i_profit = profit[i]
    # fuel deduction:
    if ((fuel_type == 'Hybrid') or (fuel_type == "Plug-In Hybrid")):
        if (counter_EV == 1):
            return i_profit
        else: 
            i_profit -= EV_fee
            return i_profit
    # import deduction
    elif (status == 1):
        if (counter_import == 1):
            return i_profit
        else:
            i_profit -= license_fee
            return i_profit
    else:
        return i_profit
    
# easy function for updating the status
def Update_EV():
    print("test1")
    global counter_EV 
    counter_EV = 1
def Update_Import():
    print("test2")
    global counter_import
    counter_import = 1

def Downdate_EV():
    global counter_EV 
    counter_EV = 0
def Downdate_Import():
    global counter_import
    counter_import = 0

# check if the car i is EV or imported
def check_EV(i):
    return ((fuel[i] == 'Hybrid') or (fuel[i] == "Plug-In Hybrid"))

def check_import(i):
    return (imported[i] == 1)

def check_constraints():
    total_size = sum([s[k] for k in items])
    total_budget = sum([budget[k] for k in items])
    if ((total_size > ParkCap) or (total_budget > Budget)):
        items.pop()
        return True
    else:
        return False

In [93]:
while True:
    # Find the most valuable item
    i = np.argmax(value_per_size)
    if value_per_size[i] == 0:
        break
    else:
        ev_record.append(check_EV(i))
        import_record.append(check_import(i))
        if (check_price(i) >= check_price(i+1)):
            items.append(i)
            # update the counter if necessary
            if check_EV(i):
                Update_EV()
            elif check_import(i):
                Update_Import()
            value_per_size[i] = 0  
            if(check_constraints()):
                should_break=True
        else:
            items.append(i+1)
            # update the counter if necessary
            if check_EV(i+1):
                Update_EV()
            elif check_import(i+1):
                Update_Import()
            value_per_size[i+1] = 0  
            if(check_constraints()):
                should_break=True
    if (should_break):
        break

test2
test2
test2
test2
test2
test2
test2
test2
test2
test2
test2
test2
test2
test2
test2
test2
test2
test2
test2
test2
test2
test2
test2
test2
test2
test2
test2
test2
test2
test2
test2
test2
test2
test2
test2
test2


In [96]:
print("Total Number of cars ready to sell: ", N)
print("Total car value: $",np.sum(profit))
print("Total Size of cars: ",size, "m^2")
print("Total buying cost amount: $", budget)
print("------------------------------------------")
print("Parking Lot Capacity: ",ParkCap, "m^2")
print("Available Budget Amount: $", Budget)


print("---------------------------------------------------------------")
print("Dealership Decision Optimizer Greedy System(D-DOGS) is working....")
print("---------------------------------------------------------------")
print(len(items), "cars sold")
print("Profit: $", sum(profit[items]))
print("Actual Size of cars sold: ", sum(s[items]), "m^2")
print("Actual budget spent: $", sum(budget[items]))

Total Number of cars ready to sell:  3223
Total car value: $ 45042160.949999996
Total Size of cars:  27387 m^2
Total buying cost amount: $ [ 7360.   24903.25 12520.   ... 35235.   41149.35 26200.  ]
------------------------------------------
Parking Lot Capacity:  16646 m^2
Available Budget Amount: $ 18721911
---------------------------------------------------------------
Dealership Decision Optimizer Greedy System(D-DOGS) is working....
---------------------------------------------------------------
88 cars sold
Profit: $ 10146820.549999999
Actual Size of cars sold:  731 m^2
Actual budget spent: $ 18668631.45


In [89]:
hybrid_or_plug_in_profit = df_new[df_new['fuel_type'].isin(['Hybrid', 'Plug-In Hybrid'])]['Profit'].mean()

# Calculate the mean profit for Gasoline cars
gasoline_profit = df_new[df_new['fuel_type'] == 'Gasoline']['Profit'].mean()

# Print the results
print("Mean profit for Hybrid or Plug-In Hybrid cars:", hybrid_or_plug_in_profit)
print("Mean profit for Gasoline cars:", gasoline_profit)

Mean profit for Hybrid or Plug-In Hybrid cars: 8294.533146067415
Mean profit for Gasoline cars: 14363.661758436943


In [91]:
hybrid_or_plug_in_profit = df_new[df_new['imported'] == 1]['Profit'].mean()

# Calculate the mean profit for Gasoline cars
gasoline_profit = df_new[df_new['imported'] == 0]['Profit'].mean()

# Print the results
print("Mean profit for Hybrid or Plug-In Hybrid cars:", hybrid_or_plug_in_profit)
print("Mean profit for Gasoline cars:", gasoline_profit)

Mean profit for Hybrid or Plug-In Hybrid cars: 14400.783993174062
Mean profit for Gasoline cars: 13620.598634812286
